In [3]:
from pyspark.context import SparkContext 
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql import Row
from pyspark.ml.linalg import DenseVector
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import os
import sys
import numpy as np
import pylab
from sklearn.neighbors import KDTree
from nltk.stem import WordNetLemmatizer
from numpy.linalg import norm
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfTransformer
reload(sys)  
sys.setdefaultencoding('utf8')

In [4]:
# Data processing
wordnet_lemmatizer = WordNetLemmatizer()

# English stop words list
stop_words = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", \
              "all", "almost", "alone", "along", "already", "also","although","always","am","among",\
              "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything",\
              "anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become",\
              "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside",\
              "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot",\
              "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down",\
              "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough",\
              "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen",\
              "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found",\
              "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he",\
              "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him",\
              "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest",\
              "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd",\
              "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most",\
              "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless",\
              "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of",\
              "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",\
              "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re",\
              "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show",\
              "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something",\
              "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that",\
              "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore",\
              "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though",\
              "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards",\
              "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we",\
              "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas",\
              "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who",\
              "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet",\
              "you", "your", "yours", "yourself", "yourselves", "the"]

# https://github.com/pararthshah/qa-memnn/blob/master/nltk_utils.py
lemmatizer = WordNetLemmatizer()

def is_noun(tag):
    return tag in ['NN', 'NNS', 'NNP', 'NNPS']

def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']

def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']

def penn_to_wn(tag):
    if is_adjective(tag):
        return wn.ADJ
    elif is_noun(tag):
        return wn.NOUN
    elif is_adverb(tag):
        return wn.ADV
    elif is_verb(tag):
        return wn.VERB
    return None

'''
takes in a list of tokens of length > 0
returns a list of the most likely part of speech for the token

'''
def get_pos(tokens):
    pos_list = nltk.pos_tag(tokens)
    #print(pos_list)
    pos_pegs = []
    for i, (token, pos) in enumerate(pos_list):        
        pos_peg = penn_to_wn(pos)
        if pos_peg is not None:
            pos_pegs.append(pos_peg)
        else:
            pos_pegs.append(None)
    return list(zip(tokens, pos_pegs))

def lemmatize(pos_tagged_tokens):
    lemmatized_tokens = []
    for i, (token, pos) in enumerate(pos_tagged_tokens):
        #print(token, pos)
        lemmatized_tokens.append(lemmatizer.lemmatize(token, pos=(pos if pos is not None else 'n')))
    return lemmatized_tokens
punctuation = ["!","%","&","(",")","+",".",":",";","<","=",">","?","*",",","\t","","\n",'"','-','/']

def buildDic(ran,filename):
    dictionary = {}
    n = 0
    for i in range(ran):
        for line in open('data/'+filename+'/' + str(i) + '.txt'):
            for ttt in range(len(line.strip().split('.')[0].split())):
                t = True
                word = line.strip().split('.')[0].split()[ttt]
                c = dealWord(word)
                if not c:
                    continue
                if c:
                    dictionary = dictionary.get(c, 0) +1       
    return dictionary

def dealWord(word):
    j = 0
    while j < len(word):
        if word[j] in punctuation:
            word = word[:j]+word[j+1:]
        else:
            j+=1
    if not word:
        return ''
    pos_tagged_tokens = get_pos([word.lower()])
    c = lemmatize(pos_tagged_tokens)[0]
    if c in nltk.corpus.stopwords.words("english"):
        return ''
    if c in stop_words:
        return ""
    return c

In [13]:
path = ["./train/neg/", "./train/pos/", "./test/neg/", "./test/pos/"]
dictionary = {}
for n in range(4):
    files_tn = os.listdir(path[n])
    for i in range(len(files_tn)):
        for line in open(path[n] + files_tn[i]):
            word_line = line.split()
            for word in word_line:
                c = dealWord(word)
                if not c:
                    continue
                if c:
                    dictionary[c] = dictionary.get(c, 0) + 1    

len(dictionary)

112104

In [34]:
# Building bag of words feature
path = ["./train/neg/", "./train/pos/"]
Key = sorted(list(dictionary.keys()), key = lambda x: dictionary[x], reverse = True)[:500]
data_all = np.zeros((20000, 500))
for n in range(2):
    files_tn = os.listdir(path[n])
    for i in range(10000):
        for line in open(path[n] + files_tn[i]):
            word_line = line.split()
            for word in word_line:
                c = dealWord(word)
                if c in Key:
                    data_all[(n == 1) * 10000 + i, Key.index(c)] += 1
                    
#pd.DataFrame(data_all).to_csv("data.csv")
data_all_df = pd.DataFrame(data_all, columns=Key)

train_data = data_all[:9000,:]
train_data = np.append(train_data, data_all[10000:19000,:], axis = 0)
validation_data = data_all[9000:10000,:]
validation_data = np.append(validation_data, data_all[19000:,:], axis = 0)
train_label = np.array([0]*9000 + [1]*9000)
test_label = np.array([0]*1000 + [1] * 1000)

In [106]:
# Process data to use Spark
train = np.append(train_data.T, train_label.reshape(1,18000), axis = 0).T
validation = np.append(validation_data.T, test_label.reshape(1,2000), axis = 0).T
np.savetxt("train3.csv", train, delimiter=",")
np.savetxt("validation3.csv", validation, delimiter=",")
train_rdd = sc.textFile("./train3.csv").map(lambda x: x.split(','))
train_df = train_rdd.toDF()
#train_df.show(10)
train_new = train_df.withColumn('_1',train_df['_1'].cast("double"))
for i in range(2,502):
    train_new = train_new.withColumn('_' + str(i),train_df['_' + str(i)].cast("double"))
test_new = test_df.withColumn('_1',test_df['_1'].cast("double"))
for i in range(2,502):
    test_new = test_new.withColumn('_' + str(i),test_df['_' + str(i)].cast("double"))
row = Row('features','label')
tr_df = train_new.rdd.map(lambda r: (row(DenseVector(r[:-1]),r[-1]))).toDF()
te_df = test_new.rdd.map(lambda r: (row(DenseVector(r[:-1]),r[-1]))).toDF()
te_df.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,2.0,2.0,1.0,...|  0.0|
|[0.0,2.0,1.0,0.0,...|  0.0|
|[3.0,1.0,0.0,0.0,...|  0.0|
|[1.0,0.0,0.0,0.0,...|  0.0|
|[2.0,3.0,0.0,1.0,...|  0.0|
+--------------------+-----+
only showing top 5 rows



In [105]:
# Random Forest in Spark
randomForest = RandomForestClassifier(numTrees=100, labelCol='label',featuresCol="features").fit(tr_df)
predictions = randomForest.transform(te_df)
evaluator = BinaryClassificationEvaluator(
    labelCol="label", rawPredictionCol="rawPrediction")
accuracy = evaluator.evaluate(predictions)
accuracy

0.8725075